In [127]:
library(bit64)
library(dplyr)
library(tokenizers)
library(stopwords)
library(tidytext)
library(tm)

Loading required package: NLP

Attaching package: 'tm'

The following object is masked from 'package:stopwords':

    stopwords



In [122]:
clean_tweets <- function(text) {
  clean_texts <- text %>%
    gsub("\\$\\w*", "", .) %>% # remove stock market tickers like $GE
    gsub("^RT\\s+", "", .) %>% # remove retweet entities
    gsub("https?://[^\\s]*", "", .) %>% # remove hyperlinks
    gsub("[[:punct:]]", "", .) %>% # remove punctuation
    #gsub("[[:digit:]]", "", .) %>% # remove digits
    gsub("#", "", .) %>% # remove hashtags
    gsub("([1-2][0-9]{3})", "ayearzzz", .) %>% # remove unnecessary spaces
    gsub("(?<![0-9a-zA-Z])[0-9]+(?![0-9a-zA-Z])", "anumberzzz", ., perl=TRUE) %>% #replace numbers with 'anumberzzz'-Token, only numbers outside of words
    tolower #
  return(clean_texts)
}


In [41]:
example_tweet = "RT 10 @Twitter @chapagain Hallo zusammen! $GE Habt einen tollen 19. Oktober 2018 und viel Spaß.  :) #good #morning http://chapagain.com.np"
clean_tweets(example_tweet) %>%
    tokenize_word_stems(stopwords = stopwords::stopwords("de"))


[[1]]
 [1] "anumberzzz" "twitter"    "chapagain"  "hallo"      "zusammen"  
 [6] "habt"       "tollen"     "anumberzzz" "oktob"      "ayearzzz"  
[11] "spaß"       "good"       "morn"

End of definitions

------------------

Start data preparation



In [104]:
importdf = read.csv('../../data/trainingssets/top_1000_emoji_tweets_03_12_18_6_labels_mapped.csv', sep=';', skip=1, header=FALSE, colClasses = c("integer64", "character", "character"), encoding="UTF-8")
  names(importdf) = c("tweet_id","tweet_full_text", "target")
  importdf = importdf[,c("tweet_id","tweet_full_text","target")]  

In [105]:
importdf

tweet_id,tweet_full_text,target
1057766705911468032,#StreamAcademy #gaming #Stream #daryl #SupportSmallStreamers @TwitchShare @TwitchRetweetr @SupStreamers @TwitchSharing @TwitchRetweet1 #FarmTogether Hier die Aufzeichnung. Vielen lieben Dank f<fc>rs zuschauen und bis zum n<e4>chsten mal https://t.co/nrI0qmpkcx,"<U+0001F60A>,<U+0001F49C>"
1057766705357877253,Ich hab einen wei<df>en Schwan gefunden!!! . Und wir hei<df>en Lena und Lisa. https://t.co/LydhD3Avtt,"<U+0001F60D>,<U+0001F60D>,<U+0001F60D>,<U+2764>"
1057766703327780865,"Gerade den Fairy Tale Movie Phoenix Priestess fertig geschaut. Ist schon lange her, dass ich bei einem Film so flennen musste. Muss ich aber beim Manga von Fairy Tail au oft genug ich liebe die Charaktere und die Serie wohl einfach zu sehr #FairyTail #Heulsuse https://t.co/CiMmpOn9Op",<U+2764>
1057766584515813378,"Das Auge wird keine Fehler sehen,wenn das Herz liebt",<U+2665>
1057766553779888128,Martinus Luther war ein Christ ! Danke f<fc>r den freien Tag Martinus,"<U+270C>,<U+0001F3FB>"
1057766509538369536,ich genie<df> jetzt noch ein sek<fc>ndchen wie gl<fc>cklich ich bin bis er mich busted und kein bock mehr auf mich hat &lt;3 sch<f6>ne tr<e4>ume w<fc>nsch ich https://t.co/qUZi8N74DX,
1057766337395728384,"Twitter Welt, auch Halloween geht zuende! Gute Nacht und schlaft alle gut! Und immer dran denken (*siehe unten) https://t.co/ybt9bdr8Xb","<U+0001F609>,<U+2764>,<U+0001F495>"
1057766239781707777,"Lena, 16 sp<fc>rt beim headbang wie ihr Hirn dagegen fliegt","<U+0001F602>,<U+0001F44D>,<U+0001F3FB>"
1057766222866145280,#MeToo (Schuppen ),"<U+0001F614>,<U+270C>,<U+0001F3FD>"
1057766197058588677,Karson Kartier,<U+0001F496>


In [106]:
y=c()
for(i in 1:nrow(importdf)) {
    #Only use first emoji per tweet for now
    y = c(y, strsplit(importdf[i, "target"], ",")[[1]][1])
}
importdf[, "target"] = y

In [160]:
multi_class_labels=c('<U+0001F602>','<U+2764>','<U+0001F4AA>')
# <U+0001F602>, <U+2764>, #U+0001F914#, U+0001F4AA, #U+0001F644#

df=importdf[!is.na(y) & y %in% multi_class_labels,]
df

,tweet_id,tweet_full_text,target
3,1057766703327780865,"Gerade den Fairy Tale Movie Phoenix Priestess fertig geschaut. Ist schon lange her, dass ich bei einem Film so flennen musste. Muss ich aber beim Manga von Fairy Tail au oft genug ich liebe die Charaktere und die Serie wohl einfach zu sehr #FairyTail #Heulsuse https://t.co/CiMmpOn9Op",<U+2764>
8,1057766239781707777,"Lena, 16 sp<fc>rt beim headbang wie ihr Hirn dagegen fliegt",<U+0001F602>
19,1057765767352147968,Ich d<fc>se nun ebenfalls ab! Bin ko! Waren anstrengende Tage. Sowohl positiv als auch Untersuchungstechnisch. Was f<fc>r ein Leben. Morgen erz<e4>hle ich neues heute gehe ich mit einem guten Gef<fc>hl hier raus! Nachti!,<U+2764>
23,1057764823126151168,Interview mit einem Vampir - Liebe #Halloween https://t.co/ydzeIngi35,<U+2764>
29,1057764167841529856,Traumhafter Ausblick nach 4 Stunden Wanderung https://t.co/UDzNXTET68,<U+2764>
31,1057764064942874624,Heute war mein 21.Geburtstag MiaSanMia @FCBayern https://t.co/CCwt9Wykl3,<U+2764>
36,1057763619654524928,Ilse hat heute bestimmt die neue Red Bull Winter Edition Rumtopf vorm Spiel getrunken,<U+0001F4AA>
37,1057763552772190208,"Ist Paris unter Macron als Stadt der Liebe noch korrekt oder sollte man sich eher wasserfeste Kleidung einpacken und das ""ertrinkende"" Venedig besuchen? Schlechte Zeiten f<fc>r alle Freunde wahrer Liebe und Verbundenheit (<fc>ber die zeitweise L<fc>sternheit hinaus). @Elea11775362",<U+2764>
42,1057762973924683776,Oh Yeah! [Bin gerade irgendwie voller Liebe bei den ganzen Herzen] HERZEN F<dc>R ALLE! https://t.co/cgKl9UiWRN,<U+2764>
46,1057762605383778304,"Happy Halloween Der Shop kann online gehen! Doch um ehrlich zu sein traue ich mich nicht haha So viel Arbeit steckt drin, viele Stunden und man k<f6>nnte unendlich vorbereiten und verbessern.. Ein paar letzte Kleinigkeiten dann gehts los https://t.co/ox7MRKbiLw",<U+2764>


In [167]:
#install.packages("VennDiagram")
library(VennDiagram)
venn.plot <- draw.triple.venn(65, 75, 85,
35, 15, 25, 5, c("First", "Second", "Third"));
grid.draw(venn.plot);
grid.newpage();


ERROR: Error in png(tf, width, height, "in", pointsize, bg, res, antialias = antialias): kann png()-Gerät nicht starten


plot without title

In [120]:
target_count = table(df[,"target"])
target_count = target_count[order(target_count)]

minority_class_len=min(target_count)
majority_classes=(names(target_count)[-1])

# Balance classes via downampling
for(label in majority_classes) {
    df = df[!(df[,"tweet_id"] %in% sample(df[df[,'target'] == label,"tweet_id"], target_count[[label]]-minority_class_len)),]
}
table(df[,"target"])


<U+0001F4AA> <U+0001F602>     <U+2764> 
           8            8            8 

In [67]:
multi_class_labels=['<U+0001F602>','<U+2764>','<U+0001F4AA>']
# <U+0001F602>, <U+2764>, #U+0001F914#, U+0001F4AA, #U+0001F644#
df=df[dfy.isin(multi_class_labels)]
dfy.dropna(inplace=True)
dfx=dfx[dfy.isin(multi_class_labels)]
dfx.dropna(inplace=True)

df=dfx.copy()
df.rename(inplace=True, columns={0: "tweet"})
df['target'] = dfy

'''# BINARY CASE: balance classes to 50:50 by dropping appropriate (randomized) fraction of majority class
majority_class='♥️'
class_freq=df['target'].value_counts()
df = df.drop(df[df['target'] == majority_class].sample(frac=(1-class_freq[1]/class_freq[0]), random_state=123).index)'''

# MULTICLASS CASE: balance classes by dropping rows from all majority classes to length of minority class
minority_class_len=(df['target'].value_counts())[-1] # set to id of minority class
majority_classes=(df['target'].value_counts()).index[0:4]
for label in majority_classes:
    df = df.drop(df[df['target'] == label].sample(n=(df['target'].value_counts().loc[label]-minority_class_len), random_state=123).index)

# prepare data for following steps
our_tweets=df['tweet'].astype(str).values.tolist()
y=df['target'].astype(str).values.tolist()

ERROR: Error: '\U' ohne Hex-Ziffern in der Zeichenkette beginnend mit ""\U" genutzt


In [128]:
corpus <- clean_tweets(df[,"tweet_full_text"]) %>%
    tokenize_word_stems(stopwords = stopwords::stopwords("de"))
corpus

[[1]]
[1] "lena"       "anumberzzz" "spfcrt"     "beim"       "headbang"  
[6] "hirn"       "dagegen"    "fliegt"    

[[2]]
[1] "menschlichkeit" "lebensrettung"  "gehen"          "immer"         
[5] "immer"          "gut"            "reagiert"       "yfcksel"       

[[3]]
[1] "ach"     "schlaft" "spe4ter" "schf6n"  "raus"    "denn"   

[[4]]
 [1] "mfcssen"               "zwei"                  "faschistoid"          
 [4] "oligarchen"            "gewe4hlt"              "liberalen"            
 [7] "kre4ft"                "gemeinsam"             "diktatur"             
[10] "verhindern"            "jederextremististmist" "demokrati"            

[[5]]
[1] "lanz"

[[6]]
[1] "dank"      "geschafft"

[[7]]
[1] "zimmer"

[[8]]
 [1] "immer"      "omen"       "dinger"     "brill"      "heut"      
 [6] "zerbrochen" "wohl"       "zerbrochen" "verrfcckt"  "ok"        
[11] "life"       "still"      "goe"        "on"        

[[9]]
 [1] "schf6n"     "resi"       "anumberzzz" "stream"     "tobi"      
 [6] "laufen"     "ganz"       "zufe4llig"  "heut"       "frfch"     
[11] "schon"      "richtig"    "shirt"      "daffcr"     "angehabt"  

[[10]]
 [1] "sorri"       "dass"        "gratisalbum" "kam"         "daher"      
 [6] "schenk"      "halt"        "anumberzzz"  "track"       "juuuuul"    

[[11]]
[1] "pokalfight" "rund"       "s04"        "mu7"       

[[12]]
[1] "hambicamp"  "findet"     "vorle4ufig" "end"        "hambi"     
[6] "geht"       "natfcrlich" "schf6n"     "war"       

[[13]]
 [1] "nanopanik"         "geffchl"           "ne4chsten"        
 [4] "wochen"            "anhalten"          "std"              
 [7] "denk"              "mal"               "fcber"            
[10] "anfang"            "erfolg"            "teilnehmen"       
[13] "nanowrimoayearzzz"

[[14]]
[1] "dank"                             "heut"                            
[3] "geburtstag"                       "gratuliert"                      
[5] "inaktivstecomunintyderwelt"       "wennichnichtstreamevergesstihrda"
[7] "ichscheidflangsamechtaufeuch"    

[[15]]
 [1] "gelacht"              "geweint"              "ge4nsehaut"          
 [4] "immer"                "sprachlo"             "regelrecht"          
 [7] "geflasht"             "ganz"                 "grodf"               
[10] "kino"                 "bohemianrhapsodi"     "bohemianrhapsodymovi"

[[16]]
 [1] "gewinnspiel"        "franzf6sisch"       "chic"              
 [4] "beautybag"          "gefe4llig"          "verliebt"          
 [7] "produkt"            "sisleyparisgermani" "deshalb"           
[10] "gleich"             "drei"               "set"               
[13] "gesichert"          "gewinnen"           "gibt"              
[16] "heut"               "je"                 "sun"               
[19] "glow"               "powder"             "inklus"            
[22] "pinsel"             "stconaiehvqkj0"    

[[17]]
 [1] "foto"           "caro234"        "etwa"           "anumberzzz"    
 [5] "wochen"         "welpenme4dchen" "aufgenommen"    "gerahmt"       
 [9] "wurd"           "heut"           "ndktweet"       "geschenkt"     
[13] "nebst"          "bfcchern"       "geburtstag"     "wurd"          
[17] "nachgefeiert"   "seid"           "toll"           "frauen"        

[[18]]
[1] "fall"      "jemand"    "msv"       "duisburg"  "sucht"     "ne4chsten"
[7] "pokalrund" "dfbpokal" 

[[19]]
[1] "aufundabkusen" "starken"       "leverkusen"    "ganz"         
[5] "stark"         "heutebmgb04"   "werkself"      "leverkusen"   
[9] "dfbpokal"     

[[20]]
[1] "barbi"    "amp"      "ham"      "sum5xp7c"

[[21]]
 [1] "ne4chst"  "rund"     "dfb"      "pokal"    "ne"       "toll"    
 [7] "stimmung" "samstag"  "endlich"  "mal"      "live"     "dabei"   
[13] "rbl"      "kommt"    "gut"      "haus"     "schlaft"  "gut"     

[[22]]
[1] "netflix"  "et"       "mon"      "sandwich"

[[23]]
 [1] "spiel"     "abhaken"   "sonntag"   "alten"     "form"      "zurfcck"  
 [7] "finden"    "geht"      "be

In [145]:
TermDocumentMatrix(sapply(corpus, paste, collapse=' '))

ERROR: Error in UseMethod("TermDocumentMatrix", x): nicht anwendbare Methode für 'TermDocumentMatrix' auf Objekt der Klasse "character" angewendet


In [154]:
text_df=data_frame(index=1:length(corpus), text=sapply(corpus, paste, collapse=' '))
text_df

index,text
1,lena anumberzzz spfcrt beim headbang hirn dagegen fliegt
2,menschlichkeit lebensrettung gehen immer immer gut reagiert yfcksel
3,ach schlaft spe4ter schf6n raus denn
4,mfcssen zwei faschistoid oligarchen gewe4hlt liberalen kre4ft gemeinsam diktatur verhindern jederextremististmist demokrati
5,lanz
6,dank geschafft
7,zimmer
8,immer omen dinger brill heut zerbrochen wohl zerbrochen verrfcckt ok life still goe on
9,schf6n resi anumberzzz stream tobi laufen ganz zufe4llig heut frfch schon richtig shirt daffcr angehabt
10,sorri dass gratisalbum kam daher schenk halt anumberzzz track juuuuul


In [159]:
library(janeaustenr)
library(tidytext)
book_words <- text_df %>%
  unnest_tokens(word, text) 

book_words

index,word
1,lena
1,anumberzzz
1,spfcrt
1,beim
1,headbang
1,hirn
1,dagegen
1,fliegt
2,menschlichkeit
2,lebensrettung


In [131]:
austen_books()

text,book
SENSE AND SENSIBILITY,Sense & Sensibility
,Sense & Sensibility
by Jane Austen,Sense & Sensibility
,Sense & Sensibility
(1811),Sense & Sensibility
,Sense & Sensibility
,Sense & Sensibility
,Sense & Sensibility
,Sense & Sensibility
CHAPTER 1,Sense & Sensibility
